In [1]:
from CPD import datasets, loss, models, train
from CPD import new_metrics_2 as metrics

%load_ext autoreload
%autoreload

import pytorch_lightning as pl
import torch.nn as nn
import torch

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
HIDDEN_LSTM = 2048 * 8 * 8
HIDDEN_LSTM = 12288
HIDDEN_SIZE = 64

class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        # Pick a pretrained model and load the pretrained weights
        model_name = "slow_r50"
        #self.extractor = torch.hub.load('facebookresearch/pytorchvideo:main', 'slow_r50', pretrained=True)
        self.extractor = torch.hub.load('facebookresearch/pytorchvideo:main', 'x3d_m', pretrained=True)
        # 5 for full cnn part
        self.extractor = nn.Sequential(*list(self.extractor.blocks[:5]))
        #self.fc1 = nn.Linear(HIDDEN_LSTM, HIDDEN_SIZE)
        self.rnn = nn.LSTM(input_size=HIDDEN_LSTM,
                           hidden_size=HIDDEN_SIZE, 
                           num_layers=1,
                           batch_first=True, 
                           dropout=0.5)
        self.fc = nn.Linear(HIDDEN_SIZE, 1)
        #self.fc1 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE // 2)
        #self.fc2 = nn.Linear(HIDDEN_SIZE // 2, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()  
        self.activation = nn.Sigmoid()        

    def forward(self, x):

        batch_size, C, timesteps, H, W = x.size()
        c_in = x 
        c_out = self.extractor(c_in) 
        r_in = c_out.transpose(1, 2).flatten(2) # batch_size, timesteps, C*H*W
        r_out = r_in
        r_out, _ = self.rnn(r_out)
        r_out = self.fc(r_out)
        #r_out = self.dropout(self.relu(self.fc1(r_out)))            
        #r_out = self.dropout(self.fc2(r_out))
        out = torch.sigmoid(r_out)
        return out    

In [4]:
%load_ext tensorboard 
%tensorboard --logdir logs --port 2021

In [5]:
def write_metrics_to_file(filename, metrics, seed):
    best_th_f1, best_time_to_FA, best_delay, auc, best_conf_matrix, best_f1, best_cover, best_th_cover, max_cover = metrics
    
    with open(filename, 'a') as f:
        f.writelines('SEED: {}\n'.format(seed))
        f.writelines('AUC: {}\n'.format(auc))
        f.writelines('Time to FA {}, delay detection {} for best-F1 threshold: {}\n'. format(round(best_time_to_FA, 4), 
                                                                                       round(best_delay, 4), 
                                                                                       round(best_th_f1, 4)))
        f.writelines('TN {}, FP {}, FN {}, TP {} for best-F1 threshold: {}\n'. format(best_conf_matrix[0],
                                                                               best_conf_matrix[1],
                                                                               best_conf_matrix[2],
                                                                               best_conf_matrix[3],
                                                                               round(best_th_f1, 4)))
        f.writelines('Max F1 {}: for best-F1 threshold {}\n'.format(round(best_f1, 4), round(best_th_f1, 4)))
        f.writelines('COVER {}: for best-F1 threshold {}\n'.format(round(best_cover, 4), round(best_th_f1, 4)))

        f.writelines('Max COVER {}: for threshold {}\n'.format(max_cover, best_th_cover))


In [19]:
threshold_number = 25
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

In [7]:
np.random.seed(42)
seeds = np.random.randint(0, 200, size=5)

In [11]:
seeds = [42]

In [12]:
for seed in seeds:
    print('CURRENT SEED: ', seed)
    models.fix_seeds(seed)

    cpd_base_model = Combine()

    for param in cpd_base_model.extractor.parameters():
        param.requires_grad = False


    cpd_model = models.CPD_model(experiments_name='explosion', 
                                 loss_type='CPD', T=8, 
                                 model=cpd_base_model, 
                                 lr=0.001, 
                                 batch_size=16, 
                                 num_workers=2)

    _ = train.train_model(model=cpd_model, max_epochs=None, experiments_name='explosion', 
                          patience=10, gpus='0', seed=seed)

    model_name = ('CPD_explosion_epoch_{}_seed_{}'.format(str(cpd_model.current_epoch), str(seed)))
    torch.save(cpd_model.state_dict(), model_name + '.pth')
    
    _, delay_list_cpd, fp_delay_list_cpd = metrics.evaluation_pipeline(cpd_model, 
                                                                       cpd_model.val_dataloader(), 
                                                                       threshold_list, 
                                                                       device='cuda:0', 
                                                                       verbose=True)    
    
    write_metrics_to_file('CPD_explosion.txt', _, seed)
    
    with open('delay_' + model_name + '.npy', 'wb') as f:
        np.save(f, delay_list_cpd)
    with open('FP_delay_' + model_name + '.npy', 'wb') as f:
        np.save(f, fp_delay_list_cpd)        
        

    models.fix_seeds(seed)

    bce_base_model = Combine()

    for param in bce_base_model.extractor.parameters():
        param.requires_grad = False


    bce_model = models.CPD_model(experiments_name='explosion', 
                                 loss_type='BCE', T=None, 
                                 model=bce_base_model, 
                                 lr=0.001, 
                                 batch_size=16, 
                                 num_workers=2)

    _ = train.train_model(model=bce_model, max_epochs=None, experiments_name='explosion', 
                          patience=10, gpus='0', seed=seed)

    model_name = ('BCE_explosion_epoch_{}_seed_{}'.format(str(bce_model.current_epoch), str(seed)))
    torch.save(bce_model.state_dict(), model_name + '.pth')

    _, delay_list_bce, fp_delay_list_bce = metrics.evaluation_pipeline(bce_model, 
                                                                       cpd_model.val_dataloader(), 
                                                                       threshold_list, 
                                                                       device='cuda:0', 
                                                                       verbose=True)

    write_metrics_to_file('BCE_explosion.txt', _, seed)

    with open('delay_' + model_name + '.npy', 'wb') as f:
        np.save(f, delay_list_bce)
    with open('FP_delay_' + model_name + '.npy', 'wb') as f:
        np.save(f, fp_delay_list_bce)        

CURRENT SEED:  42


Using cache found in /home/eromanenkova/.cache/torch/hub/facebookresearch_pytorchvideo_main
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name  | Type    | Params
----------------------------------
0 | model | Combine | 5.2 M 
1 | loss  | CPDLoss | 0     
----------------------------------
3.2 M     Trainable params
2.0 M     Non-trainable params
5.2 M     Total params
20.675    Total estimated model params size (MB)


Equal sampling is impossible, do random sampling.


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.007 >= min_delta = 0. New best score: 0.180


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
seeds = [102, 179, 92, 106, 0]

In [ ]:
for seed in seeds:
    print('CURRENT SEED: ', seed)
    models.fix_seeds(seed)

    indid_base_model = Combine()

    for param in indid_base_model.extractor.parameters():
        param.requires_grad = False

    indid_model = models.CPD_model(experiments_name='explosion', 
                                   loss_type='BCE', T=None, 
                                   model=indid_base_model, 
                                   lr=0.001, 
                                   batch_size=16, 
                                   num_workers=2)

    _ = train.train_model(model=indid_model, max_epochs=50, experiments_name='explosion', 
                          patience=None, gpus='0', seed=seed)
    
        
    indid_model = models.CPD_model(experiments_name='explosion', 
                                   loss_type='CPD', T=8, 
                                   model=indid_model.model, 
                                   lr=0.001, 
                                   batch_size=16, 
                                   num_workers=2)

    _ = train.train_model(model=indid_model, max_epochs=None, experiments_name='explosion', 
                          patience=10, gpus='0', seed=seed)

    model_name = ('INDID_explosion_epoch_{}_seed_{}'.format(str(indid_model.current_epoch), str(seed)))
    torch.save(indid_model.state_dict(), model_name + '.pth')
    
    _, delay_list_indid, fp_delay_list_indid = metrics.evaluation_pipeline(indid_model, 
                                                                           indid_model.val_dataloader(), 
                                                                           threshold_list, 
                                                                           device='cuda:0', 
                                                                           verbose=True)    
    
    write_metrics_to_file('INDID_explosion.txt', _, seed)
    
    with open('delay_' + model_name + '.npy', 'wb') as f:
        np.save(f, delay_list_cpd)
    with open('FP_delay_' + model_name + '.npy', 'wb') as f:
        np.save(f, fp_delay_list_cpd)        

CURRENT SEED:  102


Using cache found in /home/eromanenkova/.cache/torch/hub/facebookresearch_pytorchvideo_main


Equal sampling is impossible, do random sampling.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name  | Type    | Params
----------------------------------
0 | model | Combine | 5.2 M 
1 | loss  | BCELoss | 0     
----------------------------------
3.2 M     Trainable params
2.0 M     Non-trainable params
5.2 M     Total params
20.675    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]